# Assignment 1: Data Exploration

This is submission for Assignment 1 for student s3879312

Analyzing the [Stack Overflow Developer Survey](https://insights.stackoverflow.com/survey) Results 2021 dataset.


## Hypothesis:

1. Woman in Tech involves more in 2021?
2. The higher years of code the higher the compensation?
3. The compensation gaps between women and men are huge?
4. Do underrepresented countries in the survey like Vietnam tend to have less gender disparity in the Tech community?
5. Higher education to get a higher commission?
6. How does frequency participation in other online platforms affect commission?
7. What are the most common languages used by among developers?
8. What are the most common jobs among respondents?
9. What are the most common languages among respondents who work in fields related to data science?
10. What are the most common languages used by developers in Vietnam?
11. Most of the respondents are also a member in other online platforms?
12. Percentages of men higher than women who are responded to this survey are narrow?
13. The country has the highest yearly compesation is the USA?

# 💾 TASK 1.8 DATA PREPARATION

### Libraries I'll use:

In [1]:
# import libraries general libraries

# work with data in tabular representation
import pandas as pd 
# round the data in the correlation matrix
import numpy as np

# Modules for data visualization
import seaborn as sns
import matplotlib.pyplot as plt 
import plotly.graph_objects as go 
import plotly.express as px

from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode, iplot, plot #iplot() = plots the figure(fig) that is created by data and layout


# word cloud library
from wordcloud import WordCloud

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

plt.rcParams['figure.figsize'] = [6, 6]

# Ensure that our plots are shown and embedded within the Jupyter notebook itself. Without this command, sometimes plots may show up in pop-up windows
%matplotlib inline

# run a python file 
%run assignment1.py

# overwrite the style of all the matplotlib graphs
sns.set()

# ignore DeprecationWarning Eror Messages
import warnings
warnings.filterwarnings('ignore')

In [2]:
# check version of Numpy (to create a requirements.txt)
np.__version__

'1.20.1'

In [3]:
# check version of Pandas (to create a requirements.txt)
pd.__version__

'1.3.4'

In [4]:
# import main dataframe
survey_public = pd.read_csv('Data/survey_data_cleaned.csv', sep=',')

# print out first 3 lines of the dataframe
style(survey_public.head(3))

,MainBranch,Employment,Country,EdLevel,Age1stCode,LearnCode,YearsCode,YearsCodePro,DevType,OrgSize,LanguageHaveWorkedWith,LanguageWantToWorkWith,DatabaseHaveWorkedWith,DatabaseWantToWorkWith,PlatformHaveWorkedWith,PlatformWantToWorkWith,WebframeHaveWorkedWith,WebframeWantToWorkWith,MiscTechHaveWorkedWith,MiscTechWantToWorkWith,ToolsTechHaveWorkedWith,ToolsTechWantToWorkWith,NEWCollabToolsHaveWorkedWith,NEWCollabToolsWantToWorkWith,OpSys,NEWStuck,NEWSOSites,SOVisitFreq,SOAccount,SOPartFreq,SOComm,NEWOtherComms,Age,Gender,Trans,Sexuality,Ethnicity,Accessibility,MentalHealth,SurveyLength,SurveyEase,ConvertedCompYearly
0,i am a developer by profession,freelancer,slovakia,secondary school,18 - 24,"coding bootcamp;other online resources (ex: videos, blogs, etc)",5.000000,3.000000,"developer, mobile",20 to 99 employees,c++;html/css;javascript;objective-c;php;swift,swift,postgresql;sqlite,sqlite,aws,aws,laravel;symfony,react.js,.net framework;.net core / .net 5,.net core / .net 5,git,git,phpstorm;xcode,atom;xcode,macos,call a coworker or friend;visit stack overflow;go for a walk or other physical activity;google it,stack overflow,multiple times per day,yes,a few times per month or weekly,"yes, definitely",no,25-34,man,no,straight / heterosexual,white or of european descent,none of the above,none of the above,appropriate in length,easy,62268.000000
1,i am a student who is learning to code,"student, full-time",netherlands,bachelor’s degree,11 - 17,"other online resources (ex: videos, blogs, etc);school",7.000000,3.000000,"developer, full-stack",20 to 99 employees,javascript;python,python,postgresql,postgresql,aws,aws,angular;flask;vue.js,react.js,cordova,.net core / .net 5,docker;git;yarn,git,android studio;intellij;notepad++;pycharm,visual studio code,windows,visit stack overflow;google it,stack overflow,daily or almost daily,yes,daily or almost daily,"yes, definitely",no,18-24,man,no,straight / heterosexual,white or of european descent,none of the above,none of the above,appropriate in length,easy,56211.000000
2,"i am not primarily a developer, but i write code sometimes as part of my work","student, full-time",russian federation,bachelor’s degree,11 - 17,"other online resources (ex: videos, blogs, etc);online forum",5.000000,3.000000,"developer, full-stack",20 to 99 employees,assembly;c;python;r;rust,julia;python;rust,sqlite,sqlite,heroku,aws,flask,flask,numpy;pandas;tensorflow;torch/pytorch,keras;numpy;pandas;tensorflow;torch/pytorch,git,git,ipython/jupyter;pycharm;rstudio;sublime text;visual studio code,ipython/jupyter;rstudio;sublime text;visual studio code,macos,visit stack overflow;google it;watch help / tutorial videos;do other work and come back later,stack overflow;stack exchange,multiple times per day,yes,multiple times per day,"yes, definitely",yes,18-24,man,no,prefer not to say,prefer not to say,none of the above,none of the above,appropriate in length,easy,56211.000000


`survey_question` dataframe below a list of the questions and their options that were asked in the survey, provided by whoever wants the analysis. I read a seperate CSV file to reference this info that I can look at while working.

In [5]:
# This is for checking the columns in the above dataframe (survey_public) meaning, make qname become index column
survey_question = pd.read_csv('Data/survey_results_schema.csv', sep=',', index_col='qname')

#remove the top unsued rows in the questions schema
survey_question = survey_question.iloc[3:,:]

# print out first 10 lines of the dataframe
survey_question.head(3)

,qid,question,force_resp,type,selector
qname,,,,,
MainBranch,QID2,Which of the following options best describes ...,True,MC,SAVR
Employment,QID24,Which of the following best describes your cur...,False,MC,MAVR
Country,QID6,"Where do you live? <span style=""font-weight: b...",True,MC,DL


I have specified the rows to select and in the next line, I have retrieved those rows using the `loc` keyword. The indices of rows loc to get the specified rows that explain the associate with the columns' names in the `survey_public`

In [6]:
# method to find the number of columns is to use .shape, which prints out the full dimensions of the data:
print(f'Dataframe contain {survey_public.shape[0]} records and {survey_public.shape[1]} columns.') # 83439 rows and 48 columns

Dataframe contain 83438 records and 42 columns.


# Task 1.8: Categorical data
Encode categorical data into digits to fit and evaluate model.

#### Few reasons why categorical values can be difficult to deal with are:

- High cardinality (Features with a large number of levels)
- Algebraic Machine Learning models, whose input must be numerical. (Hence categorical must be transformed into numbers before applying a learning algorithm to them)
- It is difficult for an ML model to differentiate between highly different levels. For example, for an ML model, `20 to 99 employees`, `100 to 499 employees`, and `10,000 or more employees` are just **3 different levels**.

I'll work with a small part of the entire data frame to demonstrate different encoding techniques and which one I choose for this project. 

### 1.8.1 Label encoding - Ordinal Values

By converting each value in the column into a number. I use this encoding technique when the categorical feature is ordinal, **ordered categories**. Hence encoding should reflect the sequence. 

In Label encoding, each label is converted into an integer value and these integers are always between 0 and 1.  

#### a. One way of achieving this is by using pandas `cat.codes` on my categorical column.

Since this method requires me to convert string data type columns into category data types, I'll use the second method as my main

In [7]:
# cast the column into category data types
survey_public['EdLevel'] = survey_public['EdLevel'].astype("category")

# create a copy of the main dataframe
survey_public_lc_example = survey_public.copy()

# label encoding
survey_public_lc_example['EdLevel'] = survey_public_lc_example['EdLevel'].cat.codes

# print out 3 sample rows of the dataframe
style(survey_public_lc_example.sample(3))

,MainBranch,Employment,Country,EdLevel,Age1stCode,LearnCode,YearsCode,YearsCodePro,DevType,OrgSize,LanguageHaveWorkedWith,LanguageWantToWorkWith,DatabaseHaveWorkedWith,DatabaseWantToWorkWith,PlatformHaveWorkedWith,PlatformWantToWorkWith,WebframeHaveWorkedWith,WebframeWantToWorkWith,MiscTechHaveWorkedWith,MiscTechWantToWorkWith,ToolsTechHaveWorkedWith,ToolsTechWantToWorkWith,NEWCollabToolsHaveWorkedWith,NEWCollabToolsWantToWorkWith,OpSys,NEWStuck,NEWSOSites,SOVisitFreq,SOAccount,SOPartFreq,SOComm,NEWOtherComms,Age,Gender,Trans,Sexuality,Ethnicity,Accessibility,MentalHealth,SurveyLength,SurveyEase,ConvertedCompYearly
65460,i am a developer by profession,employed full-time,bangladesh,1,11 - 17,"other online resources (ex: videos, blogs, etc);school;online forum;online courses or certification;books / physical media",8.000000,1.000000,"developer, full-stack",100 to 499 employees,c++;go;java;javascript;node.js;python;sql;typescript,c++;dart;go;java;javascript;node.js;python;ruby;rust;sql;typescript,couchbase;mongodb;mysql;oracle;sqlite,couchbase;firebase;mongodb;mysql;oracle;redis,aws;digitalocean,aws;digitalocean;google cloud platform;heroku,express;jquery;react.js;ruby on rails;spring,express;react.js;ruby on rails;spring;svelte;vue.js,numpy;pandas,flutter;numpy;pandas;react native,docker;git,deno;docker;git;kubernetes;puppet;terraform,android studio;intellij;ipython/jupyter;visual studio code;webstorm,android studio;intellij;ipython/jupyter;visual studio code;webstorm,linux-based,call a coworker or friend;visit stack overflow;google it;watch help / tutorial videos,stack overflow;stack exchange,multiple times per day,yes,less than once per month or monthly,"yes, definitely",no,18-24,man,no,straight / heterosexual,south asian,none of the above,i have an anxiety disorder,appropriate in length,easy,8832.000000
49080,i am a developer by profession,employed full-time,pakistan,3,25 - 34,"other online resources (ex: videos, blogs, etc);online forum;online courses or certification",3.000000,2.000000,"developer, front-end",20 to 99 employees,bash/shell;c;c#;c++;html/css;java;javascript;node.js;php;sql;typescript,bash/shell;html/css;javascript;node.js;python;ruby;typescript,microsoft sql server,mongodb;mysql,aws,aws,angular;asp.net;asp.net core ;jquery,angular;asp.net;asp.net core ;react.js;ruby on rails;vue.js,.net framework,.net framework;.net core / .net 5;flutter,git,git,netbeans;notepad++;visual studio code,visual studio code,windows,call a coworker or friend;visit stack overflow;google it,stack overflow,daily or almost daily,yes,i have never participated in q&a on stack overflow,"yes, somewhat",no,25-34,man,yes,bisexual,south asian,none of the above,none of the above,appropriate in length,neither easy nor difficult,56211.000000
5318,i am a developer by profession,freelancer,czech republic,3,11 - 17,"other online resources (ex: videos, blogs, etc);school;online forum;books / physical media",37.000000,23.000000,"developer, desktop or enterprise applications;developer, game or graphics",freelancer,java;javascript;scala;typescript,javascript;scala;typescript,mysql,postgresql,google cloud platform,heroku,react.js,react.js,.net framework;.net core / .net 5,.net core / .net 5,git,git,intellij,intellij,windows,call a coworker or friend;google it;do other work and come back later,stack overflow;stack exchange,daily or almost daily,yes,a few times per week,"yes, somewhat",no,45-54,man,no,straight / heterosexual,white or of european descent,none of the above,none of the above,appropriate in length,easy,56211.000000


#### b. Using `scikit-learn`'s - `LabelEncoder` or `category_encoders` - I use this 

I see that I just had to use the `OrdinalEncoder()` call on my series, and it automatically makes new columns for each unique value in my series and fills them with number level as appropriate.

**Note:** I don't include `EdLevel` because I want to explain it explicitly later using another method**

In [8]:
!pip install category_encoders --quiet
import category_encoders as ce

# ordinal columns using OrdinalEncoder
cols_lc = ['Age1stCode', 'OrgSize', 'SOVisitFreq', 'SOPartFreq','SurveyLength','SurveyEase', 'Age', 'SOVisitFreq']

encoder = ce.OrdinalEncoder(cols=cols_lc,return_df=True)
# create an encoded dataframe
survey_public_encoded = survey_public.copy()

# transform on the encoded dataframe
survey_public_encoded = encoder.fit_transform(survey_public_encoded)

# print out random 3 sample rows 
style(survey_public_encoded.sample(3))

,MainBranch,Employment,Country,EdLevel,Age1stCode,LearnCode,YearsCode,YearsCodePro,DevType,OrgSize,LanguageHaveWorkedWith,LanguageWantToWorkWith,DatabaseHaveWorkedWith,DatabaseWantToWorkWith,PlatformHaveWorkedWith,PlatformWantToWorkWith,WebframeHaveWorkedWith,WebframeWantToWorkWith,MiscTechHaveWorkedWith,MiscTechWantToWorkWith,ToolsTechHaveWorkedWith,ToolsTechWantToWorkWith,NEWCollabToolsHaveWorkedWith,NEWCollabToolsWantToWorkWith,OpSys,NEWStuck,NEWSOSites,SOVisitFreq,SOAccount,SOPartFreq,SOComm,NEWOtherComms,Age,Gender,Trans,Sexuality,Ethnicity,Accessibility,MentalHealth,SurveyLength,SurveyEase,ConvertedCompYearly
70823,i am a developer by profession,"student, part-time",indonesia,college/university but no degree,1,school;online forum;friend or family member;online courses or certification;colleague,7.000000,3.000000,"developer, full-stack",1,html/css;java;javascript;node.js;php;python;sql,html/css;java;javascript;kotlin;node.js;php;python;rust;sql,firebase;mariadb;postgresql,cassandra;mariadb;postgresql,aws;digitalocean;google cloud platform;heroku,aws;digitalocean;heroku,angular;angular.js;express;laravel;vue.js,laravel,cordova;qt;tensorflow,flutter;qt,git;kubernetes;yarn,docker;git,android studio;atom;netbeans;notepad++;sublime text,android studio;atom;notepad++,other,visit stack overflow;google it;watch help / tutorial videos;meditate,stack overflow;stack exchange,-1.000000,yes,4,"yes, definitely",yes,1,man,no,prefer not to say,east asian,none of the above,i have an anxiety disorder,1,1,56211.000000
52791,i am a developer by profession,freelancer,united states of america,bachelor’s degree,1,"other online resources (ex: videos, blogs, etc)",10.000000,10.000000,"developer, full-stack;engineer, data;developer, back-end;devops specialist;engineering manager",3,bash/shell;go;html/css;javascript;node.js;python;sql,elixir;javascript;node.js;python;rust;sql,postgresql;redis,postgresql;redis,aws;google cloud platform,aws,django;express;flask;react.js,express;fastapi;flask,pandas,.net core / .net 5,docker;git;kubernetes,docker;git,vim;visual studio code,visual studio code,macos,visit stack overflow;go for a walk or other physical activity;google it,stack overflow;stack exchange,-1.000000,yes,6,"no, not really",no,1,man,no,straight / heterosexual,middle eastern,none of the above,i have an anxiety disorder,1,1,200000.000000
30792,"i am not primarily a developer, but i write code sometimes as part of my work",employed full-time,taiwan,master’s degree,1,coding bootcamp;school;friend or family member;online courses or certification;colleague;books / physical media,23.000000,14.000000,"developer, mobile;developer, front-end;other",4,c#;java;javascript;typescript,c#;javascript;python;typescript,microsoft sql server;mysql;oracle,microsoft sql server,microsoft azure,google cloud platform;microsoft azure,angular;asp.net;jquery,angular;asp.net;asp.net core,.net framework,.net framework;.net core / .net 5;cordova,git,git,netbeans;visual studio;visual studio code,android studio;visual studio;visual studio code;xcode,windows,call a coworker or friend;visit stack overflow;google it;watch help / tutorial videos;do other work and come back later,stack overflow,-1.000000,not sure/can't remember,6,"no, not really",no,5,man,no,prefer not to say,east asian,none of the above,none of the above,2,2,56211.000000


## 1.8.2 One-hot encoding - Nominal Values



#### a. using `scikit-learn`'s `LabelBinarizer` and `OneHotEncoder` or even with `category_encoders`: 
 
While one-hot encoding solves the problem of unequal weights given to categories within a feature, which is **not useful when there are many categories**, as that will result in the formation of as many new columns, which can result in the curse of dimensionality.

The concept of the **curse of dimensionality** discusses that in high-dimensional spaces some things just **stop working** properly.

**Note: I encountered "MemoryError: Unable to allocate 3.07 GiB for an array with shape (28690, 28690) and data type int32" which I could not fix by using Python 64 or casting my data type into `np.uint8`. This issue possibly resolved if I update my hardware or use distributed framework like Spark. These options are not available to me; therefore, I manually encode some of the categorical columns that needed for my further analysis by using `get_dummies()`**

In [9]:
import category_encoders as ce

cols_one_hot = ['MainBranch', 'Employment', 'OpSys', 'SOAccount', 'SOComm', 'NEWOtherComms', 'Trans', 'LearnCode', 'DevType', 'LanguageHaveWorkedWith', 'LanguageWantToWorkWith', 'DatabaseHaveWorkedWith', 
 'DatabaseWantToWorkWith', 'PlatformHaveWorkedWith', 'PlatformWantToWorkWith', 'WebframeHaveWorkedWith', 'WebframeWantToWorkWith',
 'MiscTechHaveWorkedWith', 'MiscTechWantToWorkWith', 'ToolsTechHaveWorkedWith', 'ToolsTechWantToWorkWith', 
 'NEWCollabToolsHaveWorkedWith', 'NEWCollabToolsWantToWorkWith', 'NEWStuck', 'NEWSOSites', 'Gender', 
 'Sexuality', 'Accessibility', 'MentalHealth']

# Loop through columns have option 'prefer not to say' and fill null values with that value
for col in cols_one_hot:
    survey_public[col] = survey_public[col].astype(np.uint8, errors='ignore')

# one hot encoder for nominal columns that contain few categories
encoder=ce.OneHotEncoder(cols=cols_one_hot,handle_unknown='return_nan',return_df=True,use_cat_names=True)
survey_public_encoded = encoder.fit_transform(survey_public_encoded)

# Print out 3 sample rows
style(survey_public_encoded.sample(3))

MemoryError: Unable to allocate 5.36 GiB for an array with shape (8626, 83438) and data type float64

### b. Dummy Variables

This encoding technique is used when the features are nominal - **no order**.  For example, in this dataset, the column `LanguageHaveWorkedWith` is categorical, but I can not arrange them in a sequence.

Dummy variables are created to deal with the categorical variables. Dummy variables will turn the categories per variable into its own binary identifier. 

#### For columns with semi comma as separator

+ I create a dummies in Pandas

The `LanguageHaveWorkedWith` has one column for each option that can be selected as a response. If a respondent has chosen an option, the corresponding column's value is `1`. Otherwise, it is `0`. If the respondents chose several options, they are separated by `;` which I will use `.get_dummies(sep=';')` to split them into distinct columns. [Each string in Series is split by sep and returned as a DataFrame of dummy/indicator variables](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.get_dummies.html)

**Note:** As Roelpi [mentioned in his blog](https://www.roelpeters.be/split-column-values-on-separator-create-dummies-pandas/) this method would lead to **data leakage** by creating features out of categories that are not in my training data. So if I do further Machine Learning on this encoded data set, I would filter the columns to only contain features from the training data.

In [10]:
survey_question.loc['Accessibility']['question']

'Which of the following describe you, if any? Please check all that apply. '

In [11]:
# nominal columns that are separated values by semi commas
sep_semicomma_cols = ['LearnCode', 'DevType', 'LanguageHaveWorkedWith', 'LanguageWantToWorkWith', 'DatabaseHaveWorkedWith', 
 'DatabaseWantToWorkWith', 'PlatformHaveWorkedWith', 'PlatformWantToWorkWith', 'WebframeHaveWorkedWith', 'WebframeWantToWorkWith',
 'MiscTechHaveWorkedWith', 'MiscTechWantToWorkWith', 'ToolsTechHaveWorkedWith', 'ToolsTechWantToWorkWith', 
 'NEWCollabToolsHaveWorkedWith', 'NEWCollabToolsWantToWorkWith', 'NEWStuck', 'NEWSOSites', 'Gender', 
 'Sexuality', 'Accessibility', 'MentalHealth']

# Convert categorical variable into dummy variables and add a prefix to the features

LearnCode = survey_public.copy()
LearnCode = survey_public['LearnCode'].str.get_dummies(sep=';').rename(lambda x: 'LearnCode_' + x, axis='columns')

DevType = survey_public.copy()
DevType = survey_public['DevType'].str.get_dummies(sep=';').rename(lambda x: 'DevType_' + x, axis='columns')

LanguageHaveWorkedWith = survey_public.copy()
LanguageHaveWorkedWith = survey_public['LanguageHaveWorkedWith'].str.get_dummies(sep=';').rename(lambda x: 'LanguageHaveWorkedWith_' + x, axis='columns')

LanguageWantToWorkWith = survey_public.copy()
LanguageWantToWorkWith = survey_public['LanguageWantToWorkWith'].str.get_dummies(sep=';').rename(lambda x: 'LanguageWantToWorkWith_' + x, axis='columns')

DatabaseHaveWorkedWith = survey_public.copy()
DatabaseHaveWorkedWith = survey_public['DatabaseHaveWorkedWith'].str.get_dummies(sep=';').rename(lambda x: 'DatabaseHaveWorkedWith_' + x, axis='columns')

DatabaseWantToWorkWith = survey_public.copy()
DatabaseWantToWorkWith = survey_public['DatabaseWantToWorkWith'].str.get_dummies(sep=';').rename(lambda x: 'DatabaseWantToWorkWith_' + x, axis='columns')

PlatformHaveWorkedWith = survey_public.copy()
PlatformHaveWorkedWith = survey_public['PlatformHaveWorkedWith'].str.get_dummies(sep=';').rename(lambda x: 'PlatformHaveWorkedWith_' + x, axis='columns')

PlatformWantToWorkWith = survey_public.copy()
PlatformWantToWorkWith = survey_public['PlatformWantToWorkWith'].str.get_dummies(sep=';').rename(lambda x: 'PlatformWantToWorkWith_' + x, axis='columns')

WebframeHaveWorkedWith = survey_public.copy()
WebframeHaveWorkedWith = survey_public['WebframeHaveWorkedWith'].str.get_dummies(sep=';').rename(lambda x: 'WebframeHaveWorkedWith_' + x, axis='columns')

WebframeWantToWorkWith = survey_public.copy()
WebframeWantToWorkWith = survey_public['WebframeWantToWorkWith'].str.get_dummies(sep=';').rename(lambda x: 'WebframeWantToWorkWith_' + x, axis='columns')

MiscTechHaveWorkedWith = survey_public.copy()
MiscTechHaveWorkedWith = survey_public['MiscTechHaveWorkedWith'].str.get_dummies(sep=';').rename(lambda x: 'MiscTechHaveWorkedWith_' + x, axis='columns')

MiscTechWantToWorkWith = survey_public.copy()
MiscTechWantToWorkWith = survey_public['MiscTechWantToWorkWith'].str.get_dummies(sep=';').rename(lambda x: 'MiscTechWantToWorkWith_' + x, axis='columns')

ToolsTechHaveWorkedWith = survey_public.copy()
ToolsTechHaveWorkedWith = survey_public['ToolsTechHaveWorkedWith'].str.get_dummies(sep=';').rename(lambda x: 'ToolsTechHaveWorkedWith_' + x, axis='columns')

ToolsTechWantToWorkWith = survey_public.copy()
ToolsTechWantToWorkWith = survey_public['ToolsTechWantToWorkWith'].str.get_dummies(sep=';').rename(lambda x: 'ToolsTechWantToWorkWith_' + x, axis='columns')

NEWCollabToolsHaveWorkedWith = survey_public.copy()
NEWCollabToolsHaveWorkedWith = survey_public['NEWCollabToolsHaveWorkedWith'].str.get_dummies(sep=';').rename(lambda x: 'NEWCollabToolsHaveWorkedWith_' + x, axis='columns')

NEWCollabToolsWantToWorkWith = survey_public.copy()
NEWCollabToolsWantToWorkWith = survey_public['NEWCollabToolsWantToWorkWith'].str.get_dummies(sep=';').rename(lambda x: 'NEWCollabToolsWantToWorkWith_' + x, axis='columns')

NEWStuck = survey_public.copy()
NEWStuck = survey_public['NEWStuck'].str.get_dummies(sep=';').rename(lambda x: 'NEWStuck_' + x, axis='columns')

NEWSOSites = survey_public.copy()
NEWSOSites = survey_public['NEWSOSites'].str.get_dummies(sep=';').rename(lambda x: 'NEWSOSites_' + x, axis='columns')

Gender = survey_public.copy()
Gender = survey_public['Gender'].str.get_dummies(sep=';').rename(lambda x: 'Gender_' + x, axis='columns')

Sexuality = survey_public.copy()
Sexuality = survey_public['Sexuality'].str.get_dummies(sep=';').rename(lambda x: 'Sexuality_' + x, axis='columns')

Accessibility = survey_public.copy()
Accessibility = survey_public['Accessibility'].str.get_dummies(sep=';').rename(lambda x: 'Accessibility_' + x, axis='columns')

MentalHealth = survey_public.copy()
MentalHealth = survey_public['MentalHealth'].str.get_dummies(sep=';').rename(lambda x: 'MentalHealth_' + x, axis='columns')

In [12]:
style(MentalHealth.head(3))

,MentalHealth_i have a concentration and/or memory disorder (e.g. adhd),"MentalHealth_i have a mood or emotional disorder (e.g. depression, bipolar disorder)",MentalHealth_i have an anxiety disorder,MentalHealth_i have autism / an autism spectrum disorder (e.g. asperger's),MentalHealth_none of the above,"MentalHealth_or, in your own words:",MentalHealth_prefer not to say
0,0,0,0,0,1,0,0
1,0,0,0,0,1,0,0
2,0,0,0,0,1,0,0


In [13]:
cols_no_sep_dummy = ['MainBranch', 'Employment', 'OpSys', 'SOAccount', 'SOComm', 'NEWOtherComms', 'Trans']
survey_public_encoded = pd.get_dummies(data=survey_public_encoded, columns=cols_no_sep_dummy)
style(survey_public_encoded.sample(3))

,Country,EdLevel,Age1stCode,LearnCode,YearsCode,YearsCodePro,DevType,OrgSize,LanguageHaveWorkedWith,LanguageWantToWorkWith,DatabaseHaveWorkedWith,DatabaseWantToWorkWith,PlatformHaveWorkedWith,PlatformWantToWorkWith,WebframeHaveWorkedWith,WebframeWantToWorkWith,MiscTechHaveWorkedWith,MiscTechWantToWorkWith,ToolsTechHaveWorkedWith,ToolsTechWantToWorkWith,NEWCollabToolsHaveWorkedWith,NEWCollabToolsWantToWorkWith,NEWStuck,NEWSOSites,SOVisitFreq,SOPartFreq,Age,Gender,Sexuality,Ethnicity,Accessibility,MentalHealth,SurveyLength,SurveyEase,ConvertedCompYearly,MainBranch_i am a developer by profession,MainBranch_i am a student who is learning to code,"MainBranch_i am not primarily a developer, but i write code sometimes as part of my work",MainBranch_i code primarily as a hobby,"MainBranch_i used to be a developer by profession, but no longer am",MainBranch_none of these,Employment_employed full-time,Employment_employed part-time,Employment_freelancer,"Employment_not employed, and not looking for work","Employment_not employed, but looking for work",Employment_prefer not to say,Employment_retired,"Employment_student, full-time","Employment_student, part-time",OpSys_bsd,OpSys_linux-based,OpSys_macos,OpSys_other,OpSys_windows,OpSys_wsl,SOAccount_no,SOAccount_not sure/can't remember,SOAccount_yes,SOComm_neutral,"SOComm_no, not at all","SOComm_no, not really",SOComm_not sure,"SOComm_yes, definitely","SOComm_yes, somewhat",NEWOtherComms_no,NEWOtherComms_yes,Trans_no,"Trans_or, in your own words:",Trans_prefer not to say,Trans_yes
59511,united states of america,bachelor’s degree,1,school,10.000000,3.000000,"developer, full-stack",1,c;c++;html/css;java;node.js;python;rust,python,mongodb;postgresql;sqlite,postgresql,aws,aws,angular.js,react.js,.net framework;.net core / .net 5,.net core / .net 5,git,git,intellij;visual studio code,visual studio code,call a coworker or friend;visit stack overflow;watch help / tutorial videos,stack overflow;stack exchange,-1.000000,6,1,man,straight / heterosexual,prefer not to say,none of the above,prefer not to say,1,1,56211.000000,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,1,0,0,0
37795,india,something else,2,"other online resources (ex: videos, blogs, etc);online courses or certification;books / physical media",5.000000,3.000000,student,1,html/css;javascript,html/css;javascript;node.js;typescript,firebase,firebase;mongodb;postgresql;redis,aws,aws,gatsby;react.js;vue.js,express;gatsby;react.js;svelte;vue.js,.net framework;.net core / .net 5,.net core / .net 5,git;yarn,docker;git;yarn,android studio;notepad++;visual studio code,notepad++;visual studio code,visit stack overflow;google it;watch help / tutorial videos,stack overflow;stack exchange,-1.000000,6,2,man,straight / heterosexual,south asian,none of the above,none of the above,1,1,56211.000000,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0
4226,united kingdom of great britain and northern ireland,bachelor’s degree,2,"other online resources (ex: videos, blogs, etc);school;online forum",15.000000,9.000000,"developer, back-end;devops specialist",2,c;go;javascript;python;rust,haskell;rust,dynamodb;elasticsearch;postgresql;sqlite,postgresql,aws;google cloud platform;ibm cloud or watson;microsoft azure,aws,react.js,react.js,numpy;pandas,.net core / .net 5,docker;git;kubernetes;terraform;yarn,deno;docker;git;kubernetes,emacs;vim,emacs;vim,google it,stack overflow;stack exchange,-1.000000,6,1,man,straight / heterosexual,white or of european descent,none of the above,none of the above,3,1,180963.000000,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0


## 1.8.3 Binary Encoding

+ This is a combination of **one-hot encoding and Hash encoding**. In this encoding scheme, the categorical feature is first converted into numerical using an ordinal encoder. Then the numbers are transformed into a binary number. After that binary value is split into different columns. This results in **fewer columns compared to one-hot encoding.**

Binary encoding works well when there are a **high number of categories**. For example, the 2 `Country` and `Ethnicity` columns in my dataset.  

You must wonder why don't we stop after the ordinal encoding part itself, or why is a need for having to convert them into their binary equivalent only to further make columns. Here's one reason why, 

- When our **categorical column is not ordinal, and the number of categories is numerous, we can not apply only Label Encoding**, as that will lead to poor decision-making of our model. Unlike the case of ordinal columns. Having one-fourth of the total number of categories as columns in the dataframe can help in better decision-making of our model.  

Once again I have used the `category_encoders` library and its `.BinaryEncoder` function to achieve this.

**Binary encoding is a memory-efficient encoding scheme as it uses fewer features than one-hot encoding. Further, It reduces the curse of dimensionality for data with high cardinality.**

In [14]:
# columns have high number of categories
cols_binary = ['Country', 'Ethnicity']

# perform binary encoding
encoder = ce.BinaryEncoder(cols=cols_binary)
survey_public_encoded = encoder.fit_transform(survey_public_encoded)

# print out random 3 sample rows
style(survey_public_encoded.sample(3))

,Country_0,Country_1,Country_2,Country_3,Country_4,Country_5,Country_6,Country_7,EdLevel,Age1stCode,LearnCode,YearsCode,YearsCodePro,DevType,OrgSize,LanguageHaveWorkedWith,LanguageWantToWorkWith,DatabaseHaveWorkedWith,DatabaseWantToWorkWith,PlatformHaveWorkedWith,PlatformWantToWorkWith,WebframeHaveWorkedWith,WebframeWantToWorkWith,MiscTechHaveWorkedWith,MiscTechWantToWorkWith,ToolsTechHaveWorkedWith,ToolsTechWantToWorkWith,NEWCollabToolsHaveWorkedWith,NEWCollabToolsWantToWorkWith,NEWStuck,NEWSOSites,SOVisitFreq,SOPartFreq,Age,Gender,Sexuality,Ethnicity_0,Ethnicity_1,Ethnicity_2,Ethnicity_3,Ethnicity_4,Ethnicity_5,Ethnicity_6,Ethnicity_7,Ethnicity_8,Accessibility,MentalHealth,SurveyLength,SurveyEase,ConvertedCompYearly,MainBranch_i am a developer by profession,MainBranch_i am a student who is learning to code,"MainBranch_i am not primarily a developer, but i write code sometimes as part of my work",MainBranch_i code primarily as a hobby,"MainBranch_i used to be a developer by profession, but no longer am",MainBranch_none of these,Employment_employed full-time,Employment_employed part-time,Employment_freelancer,"Employment_not employed, and not looking for work","Employment_not employed, but looking for work",Employment_prefer not to say,Employment_retired,"Employment_student, full-time","Employment_student, part-time",OpSys_bsd,OpSys_linux-based,OpSys_macos,OpSys_other,OpSys_windows,OpSys_wsl,SOAccount_no,SOAccount_not sure/can't remember,SOAccount_yes,SOComm_neutral,"SOComm_no, not at all","SOComm_no, not really",SOComm_not sure,"SOComm_yes, definitely","SOComm_yes, somewhat",NEWOtherComms_no,NEWOtherComms_yes,Trans_no,"Trans_or, in your own words:",Trans_prefer not to say,Trans_yes
20585,0,0,0,0,0,1,0,1,bachelor’s degree,2,"other online resources (ex: videos, blogs, etc);school;online courses or certification",7.000000,3.000000,"developer, full-stack",1,html/css;javascript;node.js;typescript,html/css;java;javascript;node.js;rust;typescript,mysql,postgresql,aws;heroku,heroku,gatsby;react.js,express;react.js,.net framework;.net core / .net 5,.net core / .net 5,git;yarn,docker;git;yarn,visual studio code,visual studio code,visit stack overflow;google it;do other work and come back later,stack overflow,-1.000000,5,2,man,straight / heterosexual,0,0,0,0,0,0,0,1,1,none of the above,none of the above,3,1,56211.000000,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0
42996,0,0,0,0,1,0,1,1,bachelor’s degree,2,school,12.000000,8.000000,"developer, back-end",7,c#;go;html/css;javascript;php;ruby;sql,bash/shell;go;html/css;javascript;node.js;php;python;ruby;sql;typescript,dynamodb;elasticsearch;mariadb;microsoft sql server;mysql;postgresql;redis;sqlite,dynamodb;elasticsearch;mariadb;mysql;postgresql;redis,aws,aws;google cloud platform,asp.net;jquery;react.js;ruby on rails,jquery;laravel;react.js;ruby on rails,.net framework,apache spark;hadoop;tensorflow,chef;docker;git;terraform;yarn,docker;git;kubernetes;terraform;yarn,atom;eclipse;intellij;rubymine;sublime text;visual studio;visual studio code;webstorm,atom;intellij;rubymine,call a coworker or friend;visit stack overflow;go for a walk or other physical activity;google it;watch help / tutorial videos;panic,stack overflow;stack exchange,-1.000000,6,1,man,prefer not to say,0,0,0,0,0,0,1,0,1,none of the above,prefer not to say,1,1,55130.000000,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0
19243,0,0,0,0,1,0,1,1,master’s degree,1,online courses or certification,8.000000,5.000000,"developer, back-end",2,c;c++;python;rust,rust;scala,postgresql;redis;sqlite,postgresql,aws;google cloud platform,aws,react.js,react.js,.net framework;.net core / .net 5,.net core / .net 5,docker;git,git,emacs;neovim;vim;visual studio code,emacs;visual studio code,call a coworker or friend;go for a walk or other physical activity;google it;do other work and come back later;meditate,stack overflow;stack exchange,-1.000000,6,1,prefer not to say,prefer not to say,0,0,0,0,0,

## 1.8.4 Replacing Values - For All Type of Categorical Values

This is one of the most basic approaches and very straightforward to implement. We can just replace the categories with the desired numbers by putting them all together. This can easily be done with the `replace()` function in pandas. The idea is that we have the liberty to choose whatever numbers we want to assign to the categories according to the business use case. 

We first create a dictionary that will contain mapping numbers(i.e the replacement) for the categories in our category column.

  **I will apply this encoding technique to the `'EdLevel'` column of our dataframe:**
The reason I have picked up the `OpSys` column is because it has a lower  number of categories. Which is why mapping is possible via a hard coded dictionary. When category count is high, typing out each mapping is not possible, which is why I cant use this technique with several categories. 

In [15]:
replace_map = {'EdLevel': {'primary/elementary school': 1, 'secondary school': 2, 'college/university but no degree': 3, 'associate degree': 4, 'bachelor’s degree': 5, 'master’s degree': 6, 'professional degree': 7, 'other doctoral degree': 8, 'something else': 9}}

# print out to see the 'EdLevel' column encoded as the mapped numbers in the dataframe above.
survey_public_encoded.replace(replace_map, inplace=True)

# print out sample 3 rows
style(survey_public_encoded.sample(3))

,Country_0,Country_1,Country_2,Country_3,Country_4,Country_5,Country_6,Country_7,EdLevel,Age1stCode,LearnCode,YearsCode,YearsCodePro,DevType,OrgSize,LanguageHaveWorkedWith,LanguageWantToWorkWith,DatabaseHaveWorkedWith,DatabaseWantToWorkWith,PlatformHaveWorkedWith,PlatformWantToWorkWith,WebframeHaveWorkedWith,WebframeWantToWorkWith,MiscTechHaveWorkedWith,MiscTechWantToWorkWith,ToolsTechHaveWorkedWith,ToolsTechWantToWorkWith,NEWCollabToolsHaveWorkedWith,NEWCollabToolsWantToWorkWith,NEWStuck,NEWSOSites,SOVisitFreq,SOPartFreq,Age,Gender,Sexuality,Ethnicity_0,Ethnicity_1,Ethnicity_2,Ethnicity_3,Ethnicity_4,Ethnicity_5,Ethnicity_6,Ethnicity_7,Ethnicity_8,Accessibility,MentalHealth,SurveyLength,SurveyEase,ConvertedCompYearly,MainBranch_i am a developer by profession,MainBranch_i am a student who is learning to code,"MainBranch_i am not primarily a developer, but i write code sometimes as part of my work",MainBranch_i code primarily as a hobby,"MainBranch_i used to be a developer by profession, but no longer am",MainBranch_none of these,Employment_employed full-time,Employment_employed part-time,Employment_freelancer,"Employment_not employed, and not looking for work","Employment_not employed, but looking for work",Employment_prefer not to say,Employment_retired,"Employment_student, full-time","Employment_student, part-time",OpSys_bsd,OpSys_linux-based,OpSys_macos,OpSys_other,OpSys_windows,OpSys_wsl,SOAccount_no,SOAccount_not sure/can't remember,SOAccount_yes,SOComm_neutral,"SOComm_no, not at all","SOComm_no, not really",SOComm_not sure,"SOComm_yes, definitely","SOComm_yes, somewhat",NEWOtherComms_no,NEWOtherComms_yes,Trans_no,"Trans_or, in your own words:",Trans_prefer not to say,Trans_yes
8718,0,0,0,0,0,1,0,1,5,2,"other online resources (ex: videos, blogs, etc);school",12.000000,2.000000,"developer, back-end",1,perl,javascript;perl;python,mysql,mysql;sqlite,aws,aws;digitalocean,react.js,react.js,.net framework;.net core / .net 5,.net core / .net 5,git,git,vim;visual studio code,vim,visit stack overflow;google it;meditate,stack overflow,-1.000000,6,2,man,straight / heterosexual,0,0,0,0,0,0,0,0,1,none of the above,none of the above,1,1,38778.000000,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,1,0,1,0,0,0
53689,0,0,0,1,0,1,0,1,6,3,"other online resources (ex: videos, blogs, etc);online forum;online courses or certification;books / physical media",8.000000,8.000000,"developer, desktop or enterprise applications;developer, full-stack;engineer, data;devops specialist",2,bash/shell;java;javascript;python;scala;sql;typescript,f#;rust;scala;sql,postgresql;sqlite,postgresql;sqlite,aws;google cloud platform,google cloud platform,react.js,react.js,apache spark,apache spark,docker;git;kubernetes;pulumi;terraform,docker;git;kubernetes;pulumi,intellij;vim;visual studio,intellij;vim;visual studio,go for a walk or other physical activity;google it;meditate,stack overflow,-1.000000,6,3,man,straight / heterosexual,0,0,0,0,0,0,0,0,1,none of the above,none of the above,1,1,82620.000000,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0
40950,0,0,0,1,0,0,0,0,4,2,"other online resources (ex: videos, blogs, etc)",5.000000,3.000000,"developer, full-stack",1,crystal;javascript;typescript,crystal;javascript;typescript,mysql,postgresql,aws,aws,svelte;vue.js,svelte;vue.js,.net framework;.net core / .net 5,.net core / .net 5,docker;git,docker;git,visual studio code,sublime text,visit stack overflow;google it,stack overflow;stack exchange,-1.000000,6,1,man,prefer not to say,0,0,0,0,0,0,0,1,0,prefer not to say,prefer not to say,2,2,56211.000000,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0


# [Merging multiple of encoded data frames](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html)

In [16]:
# concat multiple data frames column-wise
encoded_data_frames = [survey_public_encoded, LearnCode, DevType, LanguageHaveWorkedWith, LanguageWantToWorkWith, 
                       DatabaseHaveWorkedWith, DatabaseWantToWorkWith, PlatformHaveWorkedWith, PlatformWantToWorkWith, 
                       WebframeHaveWorkedWith, WebframeWantToWorkWith, MiscTechHaveWorkedWith, MiscTechWantToWorkWith, 
                       ToolsTechHaveWorkedWith, ToolsTechWantToWorkWith, NEWCollabToolsHaveWorkedWith, 
                       NEWCollabToolsWantToWorkWith, NEWStuck, NEWSOSites, Gender, Sexuality, Accessibility, MentalHealth]
encoded_data = pd.concat(encoded_data_frames, axis=1)
style(encoded_data.sample(3))

,Country_0,Country_1,Country_2,Country_3,Country_4,Country_5,Country_6,Country_7,EdLevel,Age1stCode,LearnCode,YearsCode,YearsCodePro,DevType,OrgSize,LanguageHaveWorkedWith,LanguageWantToWorkWith,DatabaseHaveWorkedWith,DatabaseWantToWorkWith,PlatformHaveWorkedWith,PlatformWantToWorkWith,WebframeHaveWorkedWith,WebframeWantToWorkWith,MiscTechHaveWorkedWith,MiscTechWantToWorkWith,ToolsTechHaveWorkedWith,ToolsTechWantToWorkWith,NEWCollabToolsHaveWorkedWith,NEWCollabToolsWantToWorkWith,NEWStuck,NEWSOSites,SOVisitFreq,SOPartFreq,Age,Gender,Sexuality,Ethnicity_0,Ethnicity_1,Ethnicity_2,Ethnicity_3,Ethnicity_4,Ethnicity_5,Ethnicity_6,Ethnicity_7,Ethnicity_8,Accessibility,MentalHealth,SurveyLength,SurveyEase,ConvertedCompYearly,MainBranch_i am a developer by profession,MainBranch_i am a student who is learning to code,"MainBranch_i am not primarily a developer, but i write code sometimes as part of my work",MainBranch_i code primarily as a hobby,"MainBranch_i used to be a developer by profession, but no longer am",MainBranch_none of these,Employment_employed full-time,Employment_employed part-time,Employment_freelancer,"Employment_not employed, and not looking for work","Employment_not employed, but looking for work",Employment_prefer not to say,Employment_retired,"Employment_student, full-time","Employment_student, part-time",OpSys_bsd,OpSys_linux-based,OpSys_macos,OpSys_other,OpSys_windows,OpSys_wsl,SOAccount_no,SOAccount_not sure/can't remember,SOAccount_yes,SOComm_neutral,"SOComm_no, not at all","SOComm_no, not really",SOComm_not sure,"SOComm_yes, definitely","SOComm_yes, somewhat",NEWOtherComms_no,NEWOtherComms_yes,Trans_no,"Trans_or, in your own words:",Trans_prefer not to say,Trans_yes,LearnCode_books / physical media,LearnCode_coding bootcamp,LearnCode_colleague,LearnCode_friend or family member,LearnCode_online courses or certification,LearnCode_online forum,LearnCode_other (please specify):,"LearnCode_other online resources (ex: videos, blogs, etc)",LearnCode_school,DevType_academic researcher,DevType_data or business analyst,DevType_data scientist or machine learning specialist,DevType_database administrator,DevType_designer,"DevType_developer, back-end","DevType_developer, desktop or enterprise applications","DevType_developer, embedded applications or devices","DevType_developer, front-end","DevType_developer, full-stack","DevType_developer, game or graphics","DevType_developer, mobile","DevType_developer, qa or test",DevType_devops specialist,DevType_educator,"DevType_engineer, data","DevType_engineer, site reliability",DevType_engineering manager,DevType_marketing or sales professional,DevType_other,DevType_product manager,DevType_scientist,"DevType_senior executive (c-suite, vp, etc.)",DevType_student,DevType_system administrator,LanguageHaveWorkedWith_apl,LanguageHaveWorkedWith_assembly,LanguageHaveWorkedWith_bash/shell,LanguageHaveWorkedWith_c,LanguageHaveWorkedWith_c#,LanguageHaveWorkedWith_c++,LanguageHaveWorkedWith_clojure,LanguageHaveWorkedWith_cobol,LanguageHaveWorkedWith_crystal,LanguageHaveWorkedWith_dart,LanguageHaveWorkedWith_delphi,LanguageHaveWorkedWith_elixir,LanguageHaveWorkedWith_erlang,LanguageHaveWorkedWith_f#,LanguageHaveWorkedWith_go,LanguageHaveWorkedWith_groovy,LanguageHaveWorkedWith_haskell,LanguageHaveWorkedWith_html/css,LanguageHaveWorkedWith_java,LanguageHaveWorkedWith_javascript,LanguageHaveWorkedWith_julia,LanguageHaveWorkedWith_kotlin,LanguageHaveWorkedWith_lisp,LanguageHaveWorkedWith_matlab,LanguageHaveWorkedWith_node.js,LanguageHaveWorkedWith_objective-c,LanguageHaveWorkedWith_perl,LanguageHaveWorkedWith_php,LanguageHaveWorkedWith_powershell,LanguageHaveWorkedWith_python,LanguageHaveWorkedWith_r,LanguageHaveWorkedWith_ruby,LanguageHaveWorkedWith_rust,LanguageHaveWorkedWith_scala,LanguageHaveWorkedWith_sql,LanguageHaveWorkedWith_swift,LanguageHaveWorkedWith_typescript,LanguageHaveWorkedWith_vba,LanguageWantToWorkWith_apl,LanguageWantToWorkWith_assembly,LanguageWantToWorkWith_bash/shell,Langu

In [17]:
# Deleted original form of already encoded columns, all categorical columns now sould all be in numeric
encoded_data.drop(sep_semicomma_cols, axis=1, inplace=True)
style(encoded_data.sample(3))

,Country_0,Country_1,Country_2,Country_3,Country_4,Country_5,Country_6,Country_7,EdLevel,Age1stCode,YearsCode,YearsCodePro,OrgSize,SOVisitFreq,SOPartFreq,Age,Ethnicity_0,Ethnicity_1,Ethnicity_2,Ethnicity_3,Ethnicity_4,Ethnicity_5,Ethnicity_6,Ethnicity_7,Ethnicity_8,SurveyLength,SurveyEase,ConvertedCompYearly,MainBranch_i am a developer by profession,MainBranch_i am a student who is learning to code,"MainBranch_i am not primarily a developer, but i write code sometimes as part of my work",MainBranch_i code primarily as a hobby,"MainBranch_i used to be a developer by profession, but no longer am",MainBranch_none of these,Employment_employed full-time,Employment_employed part-time,Employment_freelancer,"Employment_not employed, and not looking for work","Employment_not employed, but looking for work",Employment_prefer not to say,Employment_retired,"Employment_student, full-time","Employment_student, part-time",OpSys_bsd,OpSys_linux-based,OpSys_macos,OpSys_other,OpSys_windows,OpSys_wsl,SOAccount_no,SOAccount_not sure/can't remember,SOAccount_yes,SOComm_neutral,"SOComm_no, not at all","SOComm_no, not really",SOComm_not sure,"SOComm_yes, definitely","SOComm_yes, somewhat",NEWOtherComms_no,NEWOtherComms_yes,Trans_no,"Trans_or, in your own words:",Trans_prefer not to say,Trans_yes,LearnCode_books / physical media,LearnCode_coding bootcamp,LearnCode_colleague,LearnCode_friend or family member,LearnCode_online courses or certification,LearnCode_online forum,LearnCode_other (please specify):,"LearnCode_other online resources (ex: videos, blogs, etc)",LearnCode_school,DevType_academic researcher,DevType_data or business analyst,DevType_data scientist or machine learning specialist,DevType_database administrator,DevType_designer,"DevType_developer, back-end","DevType_developer, desktop or enterprise applications","DevType_developer, embedded applications or devices","DevType_developer, front-end","DevType_developer, full-stack","DevType_developer, game or graphics","DevType_developer, mobile","DevType_developer, qa or test",DevType_devops specialist,DevType_educator,"DevType_engineer, data","DevType_engineer, site reliability",DevType_engineering manager,DevType_marketing or sales professional,DevType_other,DevType_product manager,DevType_scientist,"DevType_senior executive (c-suite, vp, etc.)",DevType_student,DevType_system administrator,LanguageHaveWorkedWith_apl,LanguageHaveWorkedWith_assembly,LanguageHaveWorkedWith_bash/shell,LanguageHaveWorkedWith_c,LanguageHaveWorkedWith_c#,LanguageHaveWorkedWith_c++,LanguageHaveWorkedWith_clojure,LanguageHaveWorkedWith_cobol,LanguageHaveWorkedWith_crystal,LanguageHaveWorkedWith_dart,LanguageHaveWorkedWith_delphi,LanguageHaveWorkedWith_elixir,LanguageHaveWorkedWith_erlang,LanguageHaveWorkedWith_f#,LanguageHaveWorkedWith_go,LanguageHaveWorkedWith_groovy,LanguageHaveWorkedWith_haskell,LanguageHaveWorkedWith_html/css,LanguageHaveWorkedWith_java,LanguageHaveWorkedWith_javascript,LanguageHaveWorkedWith_julia,LanguageHaveWorkedWith_kotlin,LanguageHaveWorkedWith_lisp,LanguageHaveWorkedWith_matlab,LanguageHaveWorkedWith_node.js,LanguageHaveWorkedWith_objective-c,LanguageHaveWorkedWith_perl,LanguageHaveWorkedWith_php,LanguageHaveWorkedWith_powershell,LanguageHaveWorkedWith_python,LanguageHaveWorkedWith_r,LanguageHaveWorkedWith_ruby,LanguageHaveWorkedWith_rust,LanguageHaveWorkedWith_scala,LanguageHaveWorkedWith_sql,LanguageHaveWorkedWith_swift,LanguageHaveWorkedWith_typescript,LanguageHaveWorkedWith_vba,LanguageWantToWorkWith_apl,LanguageWantToWorkWith_assembly,LanguageWantToWorkWith_bash/shell,LanguageWantToWorkWith_c,LanguageWantToWorkWith_c#,LanguageWantToWorkWith_c++,LanguageWantToWorkWith_clojure,LanguageWantToWorkWith_cobol,LanguageWantToWorkWith_crystal,LanguageWantToWorkWith_dart,LanguageWantToWorkWith_delphi,LanguageWantToWorkWith_elixir,LanguageWantToWorkWith_erlang,LanguageWantToWorkWith_f#,LanguageWantToWorkWith_go,LanguageWantToWorkWith_groovy,LanguageWantToWorkWith_haskell,LanguageWantToWorkWith_

### WRITING DATA BACK TO FILES
After completing your analysis and adding new columns, I write the results back to a file and get that data to visualize on Dash App. Otherwise, the data will be lost when the Jupyter Lab shuts down. 

In [ ]:
# To write the data from the data frame into a file, use the to_csv function.
encoded_data.to_csv('Data/encoded_data.csv', index=False)
encoded_data.to_csv('Data/encoded_data.csv', index=False)
encoded_data.to_csv('Data/encoded_data.csv', index=False)

credit: tnathu

# References
+ [Merge multiple dataframes without common columns](https://stackoverflow.com/questions/60341348/merge-multiple-dataframes-without-common-columns)
- [Other encoding techniques with category_encoder](https://contrib.scikit-learn.org/category_encoders/)
- [Categorical data with big data using pyspark](https://towardsdatascience.com/feature-encoding-with-spark-2-3-0-part-1-9ede45562740)
- [More on Categorical_encoding](https://www.analyticsvidhya.com/blog/2020/08/types-of-categorical-data-encoding/)